In [24]:
import os
from niceview.utils.convert import h5ad_converter, delete_sample
import cv2
import numpy as np
from scipy import ndimage
import scipy
from niceview.utils.tools import mask_to_image
import scanpy as sc
import pandas as pd
from niceview.utils.cell import get_nuclei_pixels
from scipy.sparse import load_npz

In [ ]:
def process_data(img_path, mask_path, adata_path):
    """Process data for big image.
    
    Args:
        img_path (str): Path to image.
        mask_path (str): Path to mask.
        adata_path (str): Path to adata.
    """
    img = cv2.imread(img_path)
    height, width, _ = img.shape
    max_dim = max(height, width)
    
    if max_dim < 10000:
        return None
    
    # calculate resize factor
    resize_factor = 10000 / max_dim
    resized_img = cv2.resize(img, (int(width * resize_factor), int(height * resize_factor)))
    
    # read mask
    mask = np.array(load_npz(mask_path).todense())
    original_shape = mask.shape
    target_shape = (int(height * resize_factor), int(width * resize_factor))  # (height, width) in numpy format

    # resize mask array
    resized_mask = ndimage.zoom(
        mask, (target_shape[0] / original_shape[0], target_shape[1] / original_shape[1]), order=0,
    )
    
    # resize adata
    adata = sc.read_h5ad(adata_path)
    adata.obsm['spatial'] = adata.obsm['spatial'] * resize_factor
    
    # write data
    cv2.imwrite('temp.png', resized_img)
    scipy.sparse.save_npz('temp.npz', scipy.sparse.csr_matrix(resized_mask))
    adata.write('temp.h5ad')

In [ ]:
# process_data('test.tiff', 'test.npz', 'test.h5ad')  # 40 seconds

In [ ]:
# then move temp files to data path and then run the following code

In [10]:
data_path = 'c:/Users/zeyuy/Dropbox/academy/github/niceview/db/data/'
db_info_path = 'c:/Users/zeyuy/Dropbox/academy/github/niceview/db/db-info.json'
cache_path = 'c:/Users/zeyuy/Dropbox/academy/github/niceview/db/cache/'
sample_id = 'big-fella'
wsi = os.path.join(data_path, 'temp.png')
h5ad_cell = os.path.join(data_path, 'temp.h5ad')
cell_mask = os.path.join(data_path, 'temp.npz')

In [7]:
delete_sample(data_path, db_info_path, sample_id)

deleted file: c:/Users/zeyuy/Dropbox/academy/github/niceview/db/data/big-fella-wsi-img.tiff


In [8]:
# h5ad_converter(
#     data_path, db_info_path, sample_id, 
#     wsi, h5ad_cell, None, cell_mask, None,
# )
# 60 seconds